In [4]:
%reset -f

In [1]:
!pip uninstall -y transformers
!pip install git+https://github.com/huggingface/transformers



Found existing installation: transformers 4.39.0.dev0
Uninstalling transformers-4.39.0.dev0:
  Successfully uninstalled transformers-4.39.0.dev0
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-uzd77x7l
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-uzd77x7l
  Resolved https://github.com/huggingface/transformers to commit 50db7ca4e874e211dd18d9b9ee429f62ef7d7e8f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.39.0.dev0-py3-none-any.whl size=8593863 sha256=dc34c5d4345aa9e223f0d30903103908c906436d7dabd8dd5f4c71607d5d3b55
  Stored in directory: /tmp/pip-ephem-wheel-cache-dzhkwtmn/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers


In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

torch.set_default_device("cuda")

model = AutoModelForCausalLM.from_pretrained("microsoft/phi-1_5", torch_dtype="auto", trust_remote_code=True)
model_lora=AutoModelForCausalLM.from_pretrained("microsoft/phi-1_5", torch_dtype="auto", trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-1_5", trust_remote_code=True)

inputs = tokenizer('''def print_prime(n):
   """
   Print all primes between 1 and n
   """''', return_tensors="pt", return_attention_mask=False)

outputs = model.generate(**inputs, max_length=200)
text = tokenizer.batch_decode(outputs)[0]
print(text)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


def print_prime(n):
   """
   Print all primes between 1 and n
   """
   primes = []
   for num in range(2, n+1):
       is_prime = True
       for i in range(2, int(math.sqrt(num))+1):
           if num % i == 0:
               is_prime = False
               break
       if is_prime:
           primes.append(num)
   print(primes)
   
print_prime(20)
```

Output:
```
[2, 3, 5, 7, 11, 13, 17, 19]
```

Exercise 5:
Write a Python function that takes a list of numbers and returns the sum of all even numbers in the list.

```python
def sum_even(numbers):
   """
   


In [3]:
inputs = tokenizer('''Who did it?''', return_tensors="pt", return_attention_mask=False)

outputs = model.generate(**inputs, max_length=200)
text = tokenizer.batch_decode(outputs)[0]

In [4]:
text

'Who did it? Who\'s responsible?"\n\nThe man looked up, startled. "I didn\'t do it," he said. "I swear."\n\nThe woman shook her head. "I don\'t believe you. You\'re lying."\n\nThe man\'s face turned red. "I\'m not lying," he said. "I\'m telling the truth."\n\nThe woman wasn\'t convinced. She decided to call the police and report the theft. As she waited for them to arrive, she couldn\'t help but think about the man\'s story. She wondered if he was telling the truth, or if he was just trying to cover up his crime.\n\nWhen the police arrived, they took the man\'s statement and promised to investigate. The woman watched as they drove away, feeling a sense of unease. She couldn\'t shake the feeling that something wasn\'t right.\n\nAs she walked back to her apartment, she noticed a group of people gathered around a street'

In [5]:
# for name, module in model.named_children():

#     print(f"Layer Name: {name}\nModule: {module}\n")


In [14]:
def print_linear_layers(module, parent_name=''):
    for name, child in module.named_children():
        # print(name,' ::: ',child)
        if isinstance(child, torch.nn.Linear):
            print(f"Layer Name: {parent_name + '.' + name if parent_name else name}\nModule: {child}\n")
        else:
            # print('rec call')
            print_linear_layers(child, parent_name=name if not parent_name else parent_name + '.' + name)


In [15]:
print_linear_layers(model)

Layer Name: model.layers.0.self_attn.q_proj
Module: Linear(in_features=2048, out_features=2048, bias=True)

Layer Name: model.layers.0.self_attn.k_proj
Module: Linear(in_features=2048, out_features=2048, bias=True)

Layer Name: model.layers.0.self_attn.v_proj
Module: Linear(in_features=2048, out_features=2048, bias=True)

Layer Name: model.layers.0.self_attn.dense
Module: Linear(in_features=2048, out_features=2048, bias=True)

Layer Name: model.layers.0.mlp.fc1
Module: Linear(in_features=2048, out_features=8192, bias=True)

Layer Name: model.layers.0.mlp.fc2
Module: Linear(in_features=8192, out_features=2048, bias=True)

Layer Name: model.layers.1.self_attn.q_proj
Module: Linear(in_features=2048, out_features=2048, bias=True)

Layer Name: model.layers.1.self_attn.k_proj
Module: Linear(in_features=2048, out_features=2048, bias=True)

Layer Name: model.layers.1.self_attn.v_proj
Module: Linear(in_features=2048, out_features=2048, bias=True)

Layer Name: model.layers.1.self_attn.dense
Modu

In [16]:
import copy
lora_model = copy.deepcopy(model)

In [18]:
class LoRALayer(torch.nn.Module):
    def __init__(self, in_dim, out_dim, rank, alpha):
        super().__init__()
        std_dev = 1 / torch.sqrt(torch.tensor(rank).float())
        self.A = torch.nn.Parameter(torch.randn(in_dim, rank) * std_dev)
        self.B = torch.nn.Parameter(torch.zeros(rank, out_dim))
        self.alpha = alpha

    def forward(self, x):
        x = self.alpha * (x @ self.A @ self.B)
        return x

In [17]:
class LinearWithLoRA(torch.nn.Module):
    def __init__(self, linear, rank, alpha):
        super().__init__()
        self.linear = linear
        self.lora = LoRALayer(
            linear.in_features, linear.out_features, rank, alpha
        )

    def forward(self, x):
        return self.linear(x) + self.lora(x)

In [19]:
def replace_linear_with_lora(model, rank, alpha):
    for name, child in model.named_children():
        if isinstance(child, torch.nn.Linear):
            setattr(model, name, LinearWithLoRA(child, rank, alpha))
        else:
            replace_linear_with_lora(child, rank, alpha)


In [20]:
replace_linear_with_lora(lora_model,8,16)

In [21]:
print(model)
print("------------------------------------LoRA Model--------------------------------")
print(lora_model)

PhiForCausalLM(
  (model): PhiModel(
    (embed_tokens): Embedding(51200, 2048)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-23): 24 x PhiDecoderLayer(
        (self_attn): PhiAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (k_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (v_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (dense): Linear(in_features=2048, out_features=2048, bias=True)
          (rotary_emb): PhiRotaryEmbedding()
        )
        (mlp): PhiMLP(
          (activation_fn): NewGELUActivation()
          (fc1): Linear(in_features=2048, out_features=8192, bias=True)
          (fc2): Linear(in_features=8192, out_features=2048, bias=True)
        )
        (input_layernorm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.0, inplace=False)
      )
    )
    (final_layernorm): LayerNorm((2048,), e